In [271]:
include("../../src/utilities.jl")
include("../../src/model_with_ribosomes_functions.jl")
include("../../src/canonical_two_stage_model_with_division_functions.jl")
using Catalyst, DifferentialEquations, Plots, Statistics, SpecialFunctions

In [272]:
function beta_3_exact(beta, gamma, rho, T)
    return (3*beta *gamma *(6 *gamma ^2*T^2*
   exp(gamma  *T)-3 *gamma ^2 *T^2-4 *gamma *T*
   exp(gamma*T)+2 *gamma *T+2*exp(gamma *
   T)-2))/(rho*(26 *gamma ^3 *T^3*
   exp(gamma*T)-13 *gamma ^3 *T^3-12 *gamma ^2*
   T^2* exp(gamma  *T)+12 *gamma ^2 *T^2-6 *gamma* 
   T *exp(gamma * T)-6 *gamma  *T+12 *exp(gamma* 
   T)-12))
end

beta_3_exact (generic function with 1 method)

In [273]:
function negative_binomial(x, mean, std)
    p = 1-mean/std^2
    r = mean^2/(std^2-mean)
    return SpecialFunctions.gamma.(x .+ r) ./ (SpecialFunctions.gamma.(x .+ 1) .* SpecialFunctions.gamma.(r)) .* (1-p) .^ r .* p .^x
end

negative_binomial (generic function with 1 method)

In [274]:
function simulate_cycle(model, p, t0, t_fin, T, species_0, saveat)
    tstops = [t0+T:T:t_fin;]
    
    prob = DiscreteProblem(model, species_0, (t0,t_fin), p)
    jump_prob = JumpProblem(model, prob, Direct(), save_positions=(false,false))
        
    condition(u,t,integrator) = t%27.5==0.0
    affect!(integrator) = integrator.u = Utilities.partition_species(integrator.u)
    cb = DiscreteCallback(condition,affect!)
    
    sol = solve(jump_prob, SSAStepper(), saveat=saveat, callback=cb, tstops=tstops)
    return sol
end

function simulate_stochastic_cycle(model, p, t0, t_fin, T, species_0, N, saveat)
   prob = DiscreteProblem(
        model,
        species_0,
        (t0,t_fin),
        p
    )
    jump_prob = JumpProblem(model, prob, Direct())
    
    function affect!(integrator)
        integrator.u = Utilities.partition_species(integrator.u)
        integrator.u[length(integrator.u)] = 0
    end
        
    condition(u,t,integrator) = u[length(u)]==N
    cb = DiscreteCallback(condition,affect!)
    sol = solve(jump_prob, SSAStepper(), callback=cb, saveat=saveat)
    pop!(sol.u)
    pop!(sol.t)
    return sol
end

simulate_stochastic_cycle (generic function with 2 methods)

## Model II

In [275]:
c2s = @reaction_network begin
        alpha,  0-->m
        beta, m-->p + m
        gamma,  m-->0
    end alpha beta gamma

Model ##ReactionSystem#377 with 3 equations
States (2):
  m(t)
  p(t)
Parameters (3):
  alpha
  beta
  gamma

In [276]:
alph = 1.0
beta = 0.2
gamma = 0.2
T=27.5


sol = simulate_cycle(c2s, [alph, beta, gamma], 0.0, 100000.0, T,
        [
            round(CanonicalTwoStageModel.mrna(alph, gamma, T, 0)),
            round(CanonicalTwoStageModel.approximate_protein_mean(alph, beta, gamma, T, 0))
        ],
        1
    )
    

retcode: Default
Interpolation: Piecewise constant interpolation
t: 107273-element Vector{Float64}:
      0.0
      1.0
      2.0
      3.0
      4.0
      5.0
      6.0
      7.0
      8.0
      9.0
     10.0
     11.0
     12.0
      ⋮
  99990.0
  99990.0
  99991.0
  99992.0
  99993.0
  99994.0
  99995.0
  99996.0
  99997.0
  99998.0
  99999.0
 100000.0
u: 107273-element Vector{Vector{Float64}}:
 [2.0, 25.0]
 [1.0, 25.0]
 [2.0, 25.0]
 [2.0, 26.0]
 [5.0, 27.0]
 [8.0, 27.0]
 [7.0, 28.0]
 [4.0, 28.0]
 [4.0, 28.0]
 [3.0, 29.0]
 [1.0, 29.0]
 [3.0, 29.0]
 [5.0, 29.0]
 ⋮
 [2.0, 30.0]
 [2.0, 30.0]
 [2.0, 32.0]
 [4.0, 32.0]
 [5.0, 32.0]
 [7.0, 32.0]
 [6.0, 33.0]
 [5.0, 38.0]
 [4.0, 39.0]
 [4.0, 42.0]
 [4.0, 42.0]
 [2.0, 42.0]

In [277]:
p = zeros(length(sol.u))
m = zeros(length(sol.u))
for i in 1:length(sol.u)
    m[i] = sol.u[i][1]
    p[i] = sol.u[i][2]

end
p

107273-element Vector{Float64}:
 25.0
 25.0
 25.0
 26.0
 27.0
 27.0
 28.0
 28.0
 28.0
 29.0
 29.0
 29.0
 29.0
  ⋮
 30.0
 30.0
 32.0
 32.0
 32.0
 32.0
 33.0
 38.0
 39.0
 42.0
 42.0
 42.0

In [278]:

x = LinRange(1,125, 125)
y = negative_binomial(
    x,
    CanonicalTwoStageModel.p_time_av(alph, beta, gamma, T),
    sqrt(CanonicalTwoStageModel.p_var_time_av(alph, beta, gamma, T))
)
histogram(p, normed=true, linecolor=:match, xlim=[0,250])
plot!(x, y)
savefig("../../data/distributions/model_II.svg")

In [279]:
print(mean(p))

36.47919793424254

In [280]:
CanonicalTwoStageModel.p_time_av(alph, beta, gamma, T)

36.70361474104823

In [281]:
var(p)

129.143540068147

In [282]:
CanonicalTwoStageModel.p_var_time_av(alph, beta, gamma, T)

124.36692709982411

## Model III

In [283]:
mwr = @reaction_network begin
    alpha,  0-->m
    beta, m + r-->p + m + r
    gamma,  m-->0
    rho, 0-->r
end alpha beta gamma rho

Model ##ReactionSystem#379 with 4 equations
States (3):
  m(t)
  r(t)
  p(t)
Parameters (4):
  alpha
  beta
  gamma
  rho

In [284]:
alph = 1.0
gamma = 0.2
rho = 0.2
T=27.5
beta = beta_3_exact(0.2, gamma, rho, T)


sol = simulate_cycle(mwr, [alph, beta, gamma, rho], 0.0, 500000.0, T,
        [
            round(ModelWithRibosomes.mrna(alph, gamma, T, 0)),
            round(ModelWithRibosomes.r(rho, T, 0)),
            round(ModelWithRibosomes.p(alph, beta, gamma, rho, T, 0)),
        ],
        2.5
    )

retcode: Default
Interpolation: Piecewise constant interpolation
t: 236363-element Vector{Float64}:
      0.0
      2.5
      5.0
      7.5
     10.0
     12.5
     15.0
     17.5
     20.0
     22.5
     25.0
     27.5
     27.5
      ⋮
 499977.5
 499977.5
 499977.5
 499980.0
 499982.5
 499985.0
 499987.5
 499990.0
 499992.5
 499995.0
 499997.5
 500000.0
u: 236363-element Vector{Vector{Float64}}:
 [2.0, 6.0, 26.0]
 [4.0, 6.0, 26.0]
 [4.0, 6.0, 29.0]
 [3.0, 6.0, 30.0]
 [4.0, 6.0, 32.0]
 [4.0, 6.0, 32.0]
 [6.0, 6.0, 34.0]
 [5.0, 8.0, 35.0]
 [4.0, 9.0, 38.0]
 [4.0, 10.0, 41.0]
 [4.0, 10.0, 43.0]
 [2.0, 10.0, 46.0]
 [2.0, 4.0, 21.0]
 ⋮
 [3.0, 18.0, 57.0]
 [0.0, 10.0, 34.0]
 [0.0, 10.0, 34.0]
 [2.0, 10.0, 35.0]
 [5.0, 10.0, 35.0]
 [6.0, 10.0, 42.0]
 [6.0, 10.0, 45.0]
 [4.0, 10.0, 46.0]
 [4.0, 10.0, 48.0]
 [2.0, 11.0, 51.0]
 [4.0, 11.0, 51.0]
 [9.0, 12.0, 55.0]

In [285]:
p = zeros(length(sol.u))
m = zeros(length(sol.u))
for i in 1:length(sol.u)
    m[i] = sol.u[i][1]
    p[i] = sol.u[i][3]

end
p

236363-element Vector{Float64}:
 26.0
 26.0
 29.0
 30.0
 32.0
 32.0
 34.0
 35.0
 38.0
 41.0
 43.0
 46.0
 21.0
  ⋮
 57.0
 34.0
 34.0
 35.0
 35.0
 42.0
 45.0
 46.0
 48.0
 51.0
 51.0
 55.0

In [286]:
x = LinRange(1,125, 125)
y = negative_binomial(
    x,
    ModelWithRibosomes.p_time_av(alph, beta, gamma, rho, T),
    sqrt(ModelWithRibosomes.p_var_time_av(alph, beta, gamma, rho, T))
)
histogram(p, normed=true, linecolor=:match, xlim=[0,250])
plot!(x, y)
savefig("../../data/distributions/model_III.svg")

## Model IV

In [289]:
slc2sm = @reaction_network begin
    alpha,  0-->m
    beta, m -->p + m
    gamma,  m-->0
    k, 0-->n
end alpha beta gamma k

alph = 1.0
gamma = 0.2
T=27.5
beta = 0.2
N = 10
k = N/T

sol = simulate_stochastic_cycle(slc2sm, [alph, beta, gamma, k], 0., 70000., T,
    [
        round(CanonicalTwoStageModel.mrna(alph, gamma, T, 0)),
        round(CanonicalTwoStageModel.approximate_protein_mean(alph, beta, gamma, T, 0)),
        0
    ],
    N,
    2.5
)

p = zeros(length(sol.u))
m = zeros(length(sol.u))
for i in 1:length(sol.u)
    m[i] = sol.u[i][1]
    p[i] = sol.u[i][2]

end
histogram(p, normed=true, linecolor=:match, xlim=[0,250])
savefig("../../data/distributions/model_IV.svg")

## Model V

In [290]:
slm = @reaction_network begin
    alpha,  0-->m
    beta, m + r-->p + m + r
    gamma,  m-->0
    rho, 0-->r
    k, 0-->n
end alpha beta gamma rho k

alph = 1.0
gamma = 0.2
rho = 0.2
T=27.5
beta = 0.97*beta_3_exact(0.2, gamma, rho, T)
N = 10
k = N/T

sol = simulate_stochastic_cycle(slm, [alph, beta, gamma, rho, k], 0., 100000., T,
    [
        round(ModelWithRibosomes.mrna(alph, gamma, T, 0)),
        round(ModelWithRibosomes.r(rho, T, 0)),
        round(ModelWithRibosomes.p(alph, beta, gamma, rho, T, 0)),
        0
    ],
    N,
    2.5
)

p = zeros(length(sol.u))
m = zeros(length(sol.u))
for i in 1:length(sol.u)
    m[i] = sol.u[i][1]
    p[i] = sol.u[i][3]

end
histogram(p, normed=true, linecolor=:match, xlim=[0,250])
savefig("../../data/distributions/model_V.svg")